# chloroplète avec Altair

Les données sont les prénoms de bébés en France de 1900 à 2019
Pour travailler avec des données géographiques, nous utiliserons geopandas, qui charge des dataframes pandas, mais avec une prise en charge des contours géographiques au format geojson. 

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

pass

# Lecture des données

Les données exportées sont au format CSV, mais avec un séparateur ; au lieu de virgules.

Les données INSEE regroupent les noms rares ou lorsque les informations au niveau du département ont été supprimées (probablement pour protéger les personnes ayant des noms peu communs ou qui étaient les seules à être nées avec ce nom au cours d'une année donnée).  Nous les supprimerons.

In [2]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
1791595,2,ALIA,2019,83,5
321851,1,CLÉMENT,1931,50,8
608543,1,GAUTHIER,2012,13,10
3423119,2,RACHEL,1975,25,20
810226,1,JEAN-LUC,1959,13,94


Ensuite, chargeons des données cartographiques des régions en France à l'aide de geopandas. 
(ps il y a d'autres referentiel possible comprenant l hexagone et les dom tom)

In [3]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
23,25,Doubs,"POLYGON ((6.80701 47.56280, 6.81666 47.54792, ..."
9,10,Aube,"POLYGON ((3.41479 48.39027, 3.42208 48.41334, ..."
1,02,Aisne,"POLYGON ((4.04797 49.40564, 4.03991 49.39740, ..."
25,27,Eure,"POLYGON ((0.29722 49.42986, 0.33898 49.44093, ..."
79,79,Deux-Sèvres,"POLYGON ((-0.89196 46.97582, -0.85592 46.97908..."



Remarquez comment depts est un dataframe geopandas. Nous l'utiliserons comme un dataframe pandas classique, mais il inclut les informations de géométrie dont nous avons besoin pour pouvoir dessiner ces régions lorsque nous les transmettons à Altair. Il suffit de s'assurer que lorsque nous travaillons avec nos données, nous les conservons dans un dataframe geopandas et non dans un dataframe ordinaire si nous voulons dessiner les départements.

Dans la cellule suivante, remarquez comment nous effectuons une fusion droite (right-merge) pour inclure les données départementales dans names. Nous effectuons cette fusion sur depts car nous avons besoin d'un dataframe geopandas. Rappelez-vous, depts est un dataframe geopandas, tandis que names est un dataframe ordinaire. Si nous effectuions une fusion à gauche (left merge) sur names, nous obtiendrions un dataframe pandas ordinaire. Après cette fusion, names et depts seront tous deux des dataframes geopandas.

Astuce: Soyez prudent lorsque vous effectuez vos jointures de données ici. Il est facile de fusionner accidentellement dans le mauvais sens et de créer accidentellement un ensemble de données beaucoup plus volumineux.

Conservez une référence au dataframe pandas ordinaire, sans données de géométrie, juste au cas où

In [4]:
# Keep a reference around to the plain pandas dataframe, without geometry data, just in case
just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')

names.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
3090158,42,Loire,"POLYGON ((3.89953 46.27591, 3.90940 46.25773, ...",2,MARION,2002,42,27
3624449,86,Vienne,"POLYGON ((-0.10212 47.06480, -0.09806 47.09135...",2,VIRGINIE,1978,86,82
2864065,60,Oise,"POLYGON ((1.78384 49.75831, 1.80898 49.75433, ...",2,LOUNA,2013,60,28
2484255,61,Orne,"POLYGON ((-0.84094 48.75222, -0.81927 48.75413...",2,GILBERTE,1936,61,15
2341870,25,Doubs,"POLYGON ((6.80701 47.56280, 6.81666 47.54792, ...",2,ENOLA,2009,25,8


# Montrer des noms 

Maintenant, choisissons un nom et examinons sa répartition au cours des 120 dernières années en France métropolitaine. Dans cet exemple, j'ai choisi le nom "Lucien", que j'aime bien pour une raison quelconque.

In [5]:
grouped = names.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum()
grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt') # Add geometry data back in
grouped

,code,nom,geometry,dpt,preusuel,sexe,nombre
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,AARON,1,160
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABBY,2,3
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDALLAH,1,7
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDEL,1,3
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDELKADER,1,3
...,...,...,...,...,...,...,...
239574,NaN,NaN,None,974,ÉSAÏE,1,3
239575,NaN,NaN,None,974,ÉTHAN,1,53
239576,NaN,NaN,None,974,ÉTIENNE,1,3
239577,NaN,NaN,None,974,ÉVA,2,32


In [6]:
pip install altair_viewer

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [7]:
name = 'LUCIEN'
subset = grouped[grouped.preusuel == name]
alt.Chart(subset).mark_geoshape(stroke='white').encode(
    tooltip=['nom', 'code', 'nombre'],
    color='nombre',
).properties(width=800, height=600)

alt.Chart(...)

# Visualisation 1 :

evolution nom bébés au cours du temps

prénoms qui sont restés populaires ou impopulaires de manière constante (top)
soudainement ou brièvement populaires ou impopulaires 

tendances dans le temps ?

In [8]:
popular_names = names.groupby(['annais', 'preusuel']).sum().reset_index()
popular_names = popular_names[popular_names['nombre'] > 1000]

top_15_names = popular_names.groupby('preusuel')['nombre'].sum().nlargest(15).index
popular_names = popular_names[popular_names['preusuel'].isin(top_15_names)]

chart = alt.Chart(popular_names).mark_bar().encode(
    x='annais:T',
    y='sum(nombre):Q',
    color='preusuel:N',
    tooltip=['preusuel:N', 'annais:T', 'sum(nombre):Q']
)

chart

alt.Chart(...)

In [9]:
emerging_names = names.groupby(['annais', 'preusuel']).sum().reset_index()
emerging_names["annee"]=emerging_names['annais']
emerging_names['annee'] = emerging_names['annee'].astype('int64')
emerging_names = emerging_names[emerging_names['annee'] > 1920]

In [10]:
emerging_names = emerging_names[emerging_names['nombre'] > 1000]

In [11]:
import altair as alt

emerging_names = emerging_names.sort_values(['annais', 'nombre'], ascending=[True, False])
emerging_names = emerging_names.groupby('preusuel').first().reset_index()

chart = alt.Chart(emerging_names).mark_line().encode(
    x='annais:T',
    y='nombre:Q',
    color=alt.condition(
        alt.datum.nombre > 5000,
        alt.value('green'),  
        alt.value('red')  
    ),
    tooltip=['preusuel:N', 'annais:T', 'nombre:Q']
)

chart


alt.Chart(...)

# Visualisation 2 

effet régional dans les données ?

Certains prénoms sont-ils plus populaires dans certaines régions ?

Les prénoms populaires le sont-ils généralement dans tout le pays ?

In [12]:
import altair as alt

top_names_region = names.groupby(['nom', 'preusuel'], as_index=False)['nombre'].sum()
top_names_region = top_names_region.groupby('nom').apply(lambda x: x.nlargest(5, 'nombre')).reset_index(drop=True)
top_names_region_grouped = depts.merge(top_names_region, how='right', left_on='nom', right_on='nom')

chart = alt.Chart(top_names_region_grouped).mark_geoshape().encode(
    x=alt.X('preusuel:N', title='Prénom'),
    y=alt.Y('nombre:Q', title='Nombre de bébés'),
    tooltip=['preusuel', 'code', 'nombre'],
    #color=alt.Color('nom:N')
    color=alt.Color('nombre')
).properties(
    width=800,
    height=600,
    title='Répartition des prénoms populaires par région'
)

chart

alt.Chart(...)

In [13]:
import altair as alt

top_names_region = names.groupby(['nom', 'preusuel'], as_index=False)['nombre'].sum()
top_names_region = top_names_region.groupby('nom').apply(lambda x: x.nlargest(5, 'nombre')).reset_index(drop=True)
top_names_region_grouped = depts.merge(top_names_region, how='right', left_on='nom', right_on='nom')

chart = alt.Chart(top_names_region_grouped).mark_geoshape().encode(
    x=alt.X('preusuel:N', title='Prénom'),
    y=alt.Y('nombre:Q', title='Nombre de bébés'),
    #color=alt.Color('nombre')
    tooltip=['preusuel', 'code', 'nombre'],
    color=alt.Color('nom:N')
).properties(
    width=800,
    height=600,
    title='Répartition des prénoms populaires par région'
)

chart

alt.Chart(...)

illisble :)

In [14]:
scatter_chart = alt.Chart(top_names_region_grouped).mark_circle().encode(
    x=alt.X('preusuel:N', title='Prénom'),
    y=alt.Y('nombre:Q', title='Nombre de bébés'),
    color=alt.Color('nom:N'),
    tooltip=['preusuel', 'code', 'nombre']
).properties(
    width=800,
    height=400,
    title='Répartition des prénoms populaires par région (Nuage de points)'
)

scatter_chart.interactive()


alt.Chart(...)

In [15]:
faceted_bar_chart = alt.Chart(top_names_region_grouped).mark_bar().encode(
    x=alt.X('preusuel:N', title='Prénom'),
    y=alt.Y('nombre:Q', title='Nombre de bébés'),
    color=alt.Color('nom:N'),
    column=alt.Column('code:N', title='Région'),
    tooltip=['preusuel', 'code', 'nombre'],
).properties(
    width=200,
    height=300,
    title='Répartition des prénoms populaires par région (Diagramme en treillis)'
).interactive()

faceted_bar_chart


alt.Chart(...)

Graphique à barres empilées montrant la répartition des prénoms populaires par région.

In [16]:
import altair as alt

top_names_region = grouped.groupby(['nom', 'preusuel'], as_index=False)['nombre'].sum()
top_names_region = top_names_region.groupby('nom').apply(lambda x: x.nlargest(5, 'nombre')).reset_index(drop=True)

chart = alt.Chart(top_names_region).mark_bar().encode(
    x=alt.X('preusuel:N', title='Prénom'),
    y=alt.Y('nombre:Q', title='Nombre de bébés'),
    color=alt.Color('nom:N')
).properties(
    width=800,
    height=400,
    title='Répartition des prénoms populaires par région'
)

chart


alt.Chart(...)

# Visualisation 3 (bonus) 

effets de genre dans les données ?
La popularité des prénoms donnés aux deux sexes évolue-t-elle de manière cohérente ?
note : sexe est une donnée binaire

### Graphique montrant l'évolution de la popularité des prénoms donnés aux deux sexes au fil des ans.

In [17]:
import altair as alt

name = 'CAMILLE'

subset = names[(names.preusuel == name)]

chart = alt.Chart(subset).mark_line().encode(
    x='annais',
    y='nombre',
    color='sexe:N',
    tooltip=['annais', 'nombre']
).properties(
    width=800,
    height=400,
    title=f"Évolution du nombre de bébés prénommés '{name}'"
)

chart

alt.Chart(...)

In [18]:
import altair as alt

gender_counts = names.groupby(['annais', 'sexe'], as_index=False)['nombre'].sum()

chart = alt.Chart(gender_counts).mark_line().encode(
    x='annais',
    y='nombre',
    color='sexe:N',
    tooltip=['annais', 'nombre']
).properties(
    width=800,
    height=400,
    title="Évolution de la popularité des prénoms par sexe"
)

chart


alt.Chart(...)